# Bài tập cuối kì Môn Học Máy
* MSSV : 18020363
* Họ và tên : Uông Viêt Dũng

# Mô tả bài toán 
Yêu cầu xây dựng mô hình để dự đoán mức độ liên quan của kết quả tìm kiếm . Ta sử dụng dữ liệu được Home Depot cung cấp bao gồm cụm từ tìm kiếm, tiêu đề sản phẩm, mô tả sản phẩm, giá trị của sản phẩm để tính độ tương quan giữa cụm từ tìm kiếm và sản phẩm.

# Sử lý dữ liệu 

### Khai báo thư viện


In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import keras as kr
import matplotlib
import xgboost
import gensim
import spacy

from time import time
import pickle
import math
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.layers import *

from gensim.utils import simple_preprocess, tokenize

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
directory = "../input/home-depot-product-search-relevance"
processed_directory = "../input/d/cshige/home-depot-dts/"

### Phân tích dữ liệu

### Dữ liệu train và test

In [ ]:
train = pd.read_csv(directory + "/train.csv.zip", encoding = "ISO-8859-1", index_col= "id")
test = pd.read_csv(directory + "/test.csv.zip", encoding = "ISO-8859-1", index_col= "id")

### Kiểm tra thông tin

In [ ]:
train.info()
test.info()

Tập train có 74067 dữ liệu và 5 cột với các mô tả như sau : 
* product_uid : mã của sản phẩm
* product_tile : tiêu đề sản phẩm
* search_term : từ khóa tìm kiếm
* relevance : mức độ tương quan 

Tập test có 166693 dữ liệu và chúng ta cần tính giá trị relevance trong tập này.

Khám phá thêm về tập train

In [ ]:
train.head(5)

Tìm hiểu về mức độ tương quan trong tập train

In [ ]:
sns.countplot(x="relevance", data= train, palette="Blues_d")
plt.show()

### Dữ liệu thuộc tính

In [ ]:
attributes=pd.read_csv(directory + "/attributes.csv.zip")

### Kiểm tra thông tin

In [ ]:
attributes.info()

Tập thuộc tính có 20440804 dữ liệu và 3 cột bao gồm :
* product_uid : mã sản phẩm.
* name : tên thuộc tính.
* value : giá trị thuộc tính

Khám phá thêm về dữ liệu thuộc tính

In [ ]:
attributes.head(10)

Nhận xét:
* product_uid : ở dưới dạng thập phân và phải chuyển về dạng số nguyên để đồng nhất dữ liệu
* name : chưa biết giá trị bullet là gì 
* value : bao gồm nhiều kí tự
* Trong 10 dữ liệu đầu ta thấy giá trị Bullet thường là 1 chuỗi các mô tả ngoài ra còn có 2 giá trị là Material mà MFG Brand Name.

Thông thường đánh giá 1 sản phẩm thì 2 giá trị nguyên liệu và nhãn hàng sẽ là giá trị để đánh giá sản phẩm. Chúng ta tìm hiểu thêm về các dữ liệu mang 2 giá trị name là Material và MFG Brand Name

In [ ]:
attributes[attributes.name == "Material"]

In [ ]:
attributes[attributes.name == "MFG Brand Name"]

Ghép các thuộc tính có chung product_uid vào chung một bảng thành phần 

In [ ]:
attributes = attributes.fillna(0)
attributes['product_uid'] = attributes['product_uid'].astype(np.int64)
attributes['String']=attributes['name'].map(str)+' '+attributes['value'].map(str)

In [ ]:
attributes1=pd.pivot_table(attributes,index=['product_uid'],values=['String'],aggfunc=lambda x: ' '.join(x))
attributes1

### Dữ liệu mô tả 

In [ ]:
product_descriptions = pd.read_csv('../input/home-depot-product-search-relevance/product_descriptions.csv.zip')

### Kiểm tra thông tin

In [ ]:
product_descriptions.info()

Tập có 124428 dữ liệu và 2 cột bao gồm : 
* product_uid : mã sản phẩm
* product_description : mô tả sản phẩm

### Clean dữ liệu

Ta thấy các dữ liệu đang hơi rời rạc, chúng ta nên nối nó lại

In [ ]:
train = pd.merge(train, product_descriptions, on='product_uid')
test = pd.merge(test, product_descriptions, on='product_uid')

In [ ]:
train.head()

Như đã nhắc tới ở trên thì trong tập dữ liệu thuộc tính có 2 giá trị quan trọng mà Material và MFG Brand Name. Chúng ta sẽ nối gái trị vào tập train và test


In [ ]:
brand_attributes = attributes[attributes.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
material_attributes = attributes[attributes.name == "Material"][["product_uid", "value"]].rename(columns={"value": "material"})
brand_attributes.drop_duplicates(subset=['product_uid'], inplace = True)
material_attributes.drop_duplicates(subset=['product_uid'], inplace = True)

# nếu có dữ liệu nào null ta sẽ để là xâu rỗng

train = pd.merge(train, brand_attributes, on='product_uid',  how='left').fillna('')
test = pd.merge(test, brand_attributes, on='product_uid',  how='left').fillna('')

train = pd.merge(train, material_attributes, on='product_uid',  how='left').fillna('')
test = pd.merge(test, material_attributes, on='product_uid',  how='left').fillna('')

train = pd.merge(train, attributes1, on='product_uid',  how='left').fillna('')
test = pd.merge(test, attributes1, on='product_uid',  how='left').fillna('')

In [ ]:
train.head()

Bước tiếp theo ta sẽ tiến hành xử lý văn bản để có được một dữ liệu sạch, làm tiền đề để huấn luyện mô hình.

Các việc cần làm để clean dữ liệu :
* Sửa các lỗi ở các trường có giá trị là object
* Bỏ hết các kí tự đặc biệt 

Code stemming tham khảo tại https://gist.github.com/susanli2016/b83d148de7394821509bd5172d2c96d3

In [ ]:
strNum = {'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}
def str_stem(s): 
    if isinstance(s, str):
        # Tách cụm bị dấu chấm ngắt giữa thành các từ
        s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([A-Z])", r"\1 \2", s)
        s = re.sub(r"([0-9])( *)\.( *)([0-9])", r"\1.\4", s)
        
        # Đổi đơn vị đo thành từ chuẩn quốc tế
        s = re.sub(r"([0-9]+)( *)(inches|inch|in|')\.?", r"\1in. ", s)
        s = re.sub(r"([0-9]+)( *)(foot|feet|ft|'')\.?", r"\1ft. ", s)
        s = re.sub(r"([0-9]+)( *)(pounds|pound|lbs|lb)\.?", r"\1lb. ", s)
        s = re.sub(r"([0-9]+)( *)(square|sq) ?\.?(feet|foot|ft)\.?", r"\1sq.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(cubic|cu) ?\.?(feet|foot|ft)\.?", r"\1cu.ft. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons|gallon|gal)\.?", r"\1gal. ", s)
        s = re.sub(r"([0-9]+)( *)(ounces|ounce|oz)\.?", r"\1oz. ", s)
        s = re.sub(r"([0-9]+)( *)(centimeters|cm)\.?", r"\1cm. ", s)
        s = re.sub(r"([0-9]+)( *)(milimeters|mm)\.?", r"\1mm. ", s)
        s = re.sub(r"([0-9]+)( *)(°|degrees|degree)\.?", r"\1 deg. ", s)
        s = re.sub(r"([0-9]+)( *)(v|volts|volt)\.?", r"\1 volt. ", s)
        s = re.sub(r"([0-9]+)( *)(wattage|watts|watt)\.?", r"\1 watt. ", s)
        s = re.sub(r"([0-9]+)( *)(amperes|ampere|amps|amp)\.?", r"\1 amp. ", s)
        s = re.sub(r"([0-9]+)( *)(qquart|quart)\.?", r"\1 qt. ", s)
        s = re.sub(r"([0-9]+)( *)(hours|hour|hrs.)\.?", r"\1 hr ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per minute|gallon per minute|gal per minute|gallons/min.|gallons/min)\.?", r"\1 gal. per min. ", s)
        s = re.sub(r"([0-9]+)( *)(gallons per hour|gallon per hour|gal per hour|gallons/hour|gallons/hr)\.?", r"\1 gal. per hr ", s)
        
        # Xóa các ký tự đặc biệt
        s = s.replace("$"," ")
        s = s.replace("?"," ")
        s = s.replace("&nbsp;"," ")
        s = s.replace("&amp;","&")
        s = s.replace("&#39;","'")
        s = s.replace("/>/Agt/>","")
        s = s.replace("</a<gt/","")
        s = s.replace("gt/>","")
        s = s.replace("/>","")
        s = s.replace("<br","")
        s = s.replace("<.+?>","")
        s = s.replace("[ &<>)(_,;:!?\+^~@#\$]+"," ")
        s = s.replace("'s\\b","")
        s = s.replace("[']+","")
        s = s.replace("[\"]+","")
        s = s.replace("-"," ")
        s = s.replace("+"," ")
        s = s.replace("..",".")
        s = s.replace(",","")
        s = s.replace("-"," ")
        s = s.replace("//","/")
        s = s.replace("/"," ")
        s = s.replace("..",".")
        s = s.replace(" / "," ")
        s = s.replace(" \\ "," ")
        s = s.replace("."," . ")
        s = s.replace("  "," ")
        s = re.sub(r"(^\.|/)", r"", s)
        s = re.sub(r"(\.|/)$", r"", s)
        s = re.sub(r"([0-9])([a-z])", r"\1 \2", s)
        s = re.sub(r"([a-z])([0-9])", r"\1 \2", s)
        s = re.sub(r"([a-z])( *)\.( *)([a-z])", r"\1 \4", s)
        s = re.sub(r"([a-z])( *)/( *)([a-z])", r"\1 \4", s)
        s = s.replace("*"," xbi ")
        s = s.replace(" by "," xbi ")

        s = (" ").join([str(strNum[z]) if z in strNum else z for z in s.split(" ")])
        s = " ".join([re.sub('[^A-Za-z0-9-./]', ' ', word) for word in s.lower().split()])
        return s
    else:
        return "null"

Sửa lỗi chính tả. Cài đặt thư viện kiểm tra chính tả

In [ ]:
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

Loại bỏ stopword, sau đó chuyển những danh/động từ về dạng nguyên gốc của từ đó, rồi tokenize câu.

In [ ]:
stop_words = set(stopwords.words('english')) 
wordnet_lemmatizer = WordNetLemmatizer()
gensim.utils.tokenize
def preprocessing(doc, correction = 0):
    words = []
    doc  = str_stem(doc)
    for word in doc.split(' '):
        if word not in stop_words:
            if correction == 1:
                if len(spell.unknown([word])):
                    word = spell.correction(word)
            word1 = wordnet_lemmatizer.lemmatize(word, pos = "n")
            word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v")
            word3 = wordnet_lemmatizer.lemmatize(word2, pos = ("a"))
            words.append(word3)
    return ' '.join(words)

Xử lý các trường có giá trị object trong tập train và test

In [ ]:
%%time

train.search_term = [preprocessing(i, correction = 1) for i in train.search_term]
train.product_description = [preprocessing(i) for i in train.product_description]
train.brand = [preprocessing(i) for i in train.brand]
train.material = [preprocessing(i) for i in train.material]
train.product_title = [preprocessing(i) for i in train.product_title]

test.search_term = [preprocessing(i, correction = 1) for i in test.search_term]
test.product_description = [preprocessing(i) for i in test.product_description]
test.brand = [preprocessing(i) for i in test.brand]
test.material = [preprocessing(i) for i in test.material]
test.product_title = [preprocessing(i) for i in test.product_title]

train.to_pickle("proccessed_train")
test.to_pickle("proccessed_test")

In [ ]:
train.head(5)

# Mô tả thuật toán, Trích xuất đặc trưng

### Ý tưởng

Đề bài là tính giá trị relevance nên ta sẽ tính độ liên quan giữa search_term và các giá trị khác, trong đó các trường bao gồm :
* Độ tương quan giữa search_term và product_title
* Độ tương quan giữa search_term và product_description
* Độ tương quan giữa search_term và String
* Sự xuất hiện của material và brand
* Độ dài của search_term


In [ ]:
#Nếu xâu rỗng thì trả về 0 còn không thì trả về 1
def str_cont(st):
    if len(st) == 0 :
        return 0
    if len(st) != 0 :
        return 1

In [ ]:
#Xử lý xâu rỗng 
train['String'] = train['String'].replace(np.nan, '' , regex=True)
train['material'] = train['material'].replace(np.nan, '' , regex=True)
train['brand'] = train['brand'].replace(np.nan, '' , regex= True)

#Trích xuất đặc trưng 
train['term_desc'] = [len(set(a.split()) & set(b.split())) for a, b in zip(train.search_term, train.product_description)]
train['term_att'] = [len(set(a.split()) & set(b.split())) for a, b in zip(train.search_term, train.String)]
train['term_prod'] = [len(set(a.split()) & set(b.split())) for a, b in zip(train.search_term, train.product_title)]
train['q_length']=[len(a.split()) for a in train['search_term']]
train['material_count'] = [str_cont(train['material'][i]) for i  in range(0, len(train['material']))]
train['brand_count'] = [str_cont(train['brand'][i]) for i  in range(0, len(train['brand']))]


In [ ]:
train

Thực hiện với tập test

In [ ]:
test['String'] = test['String'].replace(np.nan, '' , regex=True)
train['material'] = test['material'].replace(np.nan, '' , regex=True)
test['brand'] = test['brand'].replace(np.nan, '' , regex= True)

test['term_desc'] = [len(set(a.split()) & set(b.split())) for a, b in zip(test.search_term, test.product_description)]
test['term_att'] = [len(set(a.split()) & set(b.split())) for a, b in zip(test.search_term, test.String)]
test['term_prod'] = [len(set(a.split()) & set(b.split())) for a, b in zip(test.search_term, test.product_title)]
test['search_length']=[len(a.split()) for a in test['search_term']]
test['material_count'] = [str_cont(test['material'][i]) for i  in range(0, len(test['material']))]
test['brand_count'] = [str_cont(test['brand'][i]) for i  in range(0, len(test['brand']))]

In [ ]:
test

# Chọn mô hình

Lấy các trường giá trị cần thiết cho việc huấn luyện

In [ ]:
train = train.drop(['id','product_uid','product_title','search_term','product_description','brand','material','String'], axis = 1)
test = test.drop(['product_uid','product_title','search_term','product_description','brand','material','String'], axis = 1)

Lựa chọn mô hình hồi quy Ridge để huấn luyện cho tập dữ liệu train1

In [ ]:
#chia tập dữ liệu train1 thành test/train
X = train.loc[:, train.columns != 'relevance']
y = train.loc[:, train.columns == 'relevance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

#Huấn luyện mô hình
rg= Ridge(alpha=.1)
rg.fit(X_train, y_train.values.ravel())
y_pred = rg.predict(X_test)
rg_mse = mean_squared_error(y_pred, y_test)
rg_rmse = np.sqrt(rg_mse)
print('Ridge RMSE: %.4f' % rg_rmse)

Dự đoán kết quả trên tập test

In [ ]:
y_pred = rg.predict(test)
y_pred

# Lưu và xuất kết quả

In [ ]:
results=pd.DataFrame()
results['id']=[i for i in ]
results['relevance']=y_pred
results.to_csv("home_depot.csv", index=False)

In [ ]:
results